In [1]:
import jsonlines
from tqdm import tqdm

with jsonlines.open('output_2.jsonl', mode='r') as writer:
    all_users = list(writer)
    print(all_users[0])

{'summary': 'Senior Technical Specialist with extensive experience in Software Development, DevOps, ProdOps and Data Warehousing. Proficient in Guidewire Applications Suite, Oracle Databases, and Business Intelligence applications. Experience in Cloud technologies (AWS Services, PCF). Champions Agile principles and methodologies. Strong leadership and stakeholders relation management skills. ', 'industryName': 'Computer Software', 'lastName': 'Sibalija', 'locationName': 'Canada', 'student': False, 'geoCountryName': 'Canada', 'geoCountryUrn': 'urn:li:fs_geo:101174742', 'geoLocationBackfilled': False, 'elt': False, 'industryUrn': 'urn:li:fs_industry:4', 'firstName': 'Darko', 'entityUrn': 'urn:li:fs_profile:ACoAAABW4qIByMMkdmkXL7f6ko5BgkKhMf_Nm3Q', 'geoLocation': {'geoUrn': 'urn:li:fs_geo:90009551'}, 'geoLocationName': 'Greater Toronto Area, Canada', 'location': {'basicLocation': {'countryCode': 'ca'}}, 'headline': 'Senior Application Support Analyst at RBC Technology & Operations', 'disp

In [2]:
print(len(all_users))

3259


In [54]:
def format_profile(profile):
    if not profile: return None
    headline = profile["headline"].strip() if "headline" in profile and profile["headline"] else "Not listed"
    industryName = profile["industryName"].strip() if "industryName" in profile and profile["industryName"] else "Not listed"
    name = profile["firstName"] + " " + profile["lastName"]
    country = profile["locationName"].strip() if "locationName" in profile and profile["locationName"] else "Not listed"
    summary = profile["summary"].strip() if "summary" in profile and profile["summary"] else "Not listed"
    experience = profile["experience"] if "experience" in profile and profile["experience"] else None
    volunteering = profile["volunteer"] if "volunteer" in profile and profile["volunteer"] else None
    education = profile["education"] if "education" in profile and profile["education"] else None
    awards = profile["honors"] if "honors" in profile else None

    def formatList(l, keys):
        res = ""
        for item_num, item in enumerate(l):
            for i, key in enumerate(keys.keys()):
                if (key not in item or not item[key]): continue
                # res += "" if i != 0 else ""
                res += keys[key] + ": " + item[key].strip() + ("\n" if i != len(keys) - 1 else "")
            if item_num != len(l) - 1:
                res += "\n"
        return res
    
    def formatExperience(experience, keys):
        res = []
        for item in experience:
            temp = ""
            for i, key in enumerate(keys.keys()):
                if (key not in item or not item[key]): continue
                # res += "" if i != 0 else ""
                temp += keys[key] + ": " + item[key].strip() + "\n"
            time_period_valid = item['timePeriod'] and 'startDate' in item['timePeriod'] and item['timePeriod']['startDate']
            month_valid = time_period_valid and 'month' in item['timePeriod']['startDate'] and item['timePeriod']['startDate']['month']
            year_valid = time_period_valid and 'year' in item['timePeriod']['startDate'] and item['timePeriod']['startDate']['year']
            date_str = f"{item['timePeriod']['startDate']['month']:0>2}/{item['timePeriod']['startDate']['year']}"if time_period_valid and month_valid and year_valid else "Not listed"
            temp += f"Starting Date: " + date_str
            res.append((temp,
                        int(item['timePeriod']['startDate']['year']) if time_period_valid and year_valid else None,
                        int(item['timePeriod']['startDate']['month']) if time_period_valid and month_valid else None))
        
        return list(reversed(res))


    formatted_education = formatList(education, { 'schoolName': 'School name', 'description': 'Description' }) if education else "Not listed"
    formatted_volunteering = formatList(volunteering, { 'companyName': 'Company name', 'role': 'Role', 'description': 'Description' }) if volunteering else "Not listed"
    formatted_awards = formatList(awards, { 'title': 'Title', 'issuer': 'Issuer', 'description': "Description" }) if awards else "Not listed"
    formatted_experience = formatExperience(experience, { 'companyName': 'Company name', 'title': 'Title', 'description': 'Description' }) if experience else []

    formatting_string = """Name: {name}

Industry: {industryName}
Country: {country}
Headline: {headline}
Summary: {summary}
---
Volunteering:
{formatted_volunteering}
---
Honors & Awards:
{formatted_awards}
---
Education:
{formatted_education}
"""
    return (formatting_string.format(name=name, country=country, industryName=industryName,
                                    headline=headline, summary=summary, 
                                    formatted_volunteering=formatted_volunteering, 
                                    formatted_education=formatted_education, formatted_awards=formatted_awards), formatted_experience)


In [11]:
real_users = [user for user in all_users if user]

In [12]:
from datasets import Dataset
ds = Dataset.from_list(real_users)

In [66]:
def map_formatting(examples):
    new_examples = []
    unbatched = []
    for i in range(len(examples["firstName"])):
        unbatched.append({
            k: v[i] for k, v in examples.items()
        })

    texts = []
    labels = []
    for example in unbatched:
        text, experiences = format_profile(example)
        if len(experiences) <= 0: continue
        experience_to_give = experiences[:-1]
        experience_label = experiences[-1]

        texts.append(text + "\n" + "\n".join(e[0] for e in experience_to_give))
        labels.append(experience_label[0])
    
    return {"input": texts, "output": labels}

new_ds = ds.map(map_formatting, remove_columns=ds.column_names, batched=True, batch_size=10)
new_ds = new_ds.add_column("instruction", ["Predict what the person's next prestigious experience on LinkedIn is going to be."]*len(new_ds["input"]))

Map:   0%|          | 0/3087 [00:00<?, ? examples/s]

In [67]:
new_ds.to_json("formatted.jsonl", orient="records", lines=True)

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

7067104